In [56]:
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Configuration de la SparkSession pour se connecter au cluster
spark = SparkSession.builder \
    .master("spark://pyspark-master:7077") \
    .appName("MySparkApp") \
    .config("spark.executor.memory", "1g") \
    .config("spark.jars", "/usr/local/spark/jars/postgresql-42.6.0.jar") \
    .getOrCreate()

In [57]:
import pandas as pd
from sqlalchemy import create_engine
# Charger la table PostgreSQL dans PySpark
df_election = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://pg-ds-dellstore:5432/us_election_db") \
    .option("dbtable", "election_data") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .load()

# Filtrer les IDs pairs pour le traitement batch
df_batch = df_election.filter(F.col("id") % 2 == 0)

# Paramètres de connexion PostgreSQL
user = "postgres"
password = "postgres"
host = "pg-ds-dellstore"
port = "5432"
database = "us_election_db"

# Créer un moteur SQLAlchemy pour PostgreSQL
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

# 1. Charger les données PostgreSQL dans un DataFrame
query = "SELECT * FROM election_data WHERE MOD(id, 2) != 0;"  # Filtrer les IDs impairs directement dans la requête
df = pd.read_sql_query(query, engine)

# 2. Sauvegarder les données en CSV
csv_file = "/tmp/election_stream.csv"
df.to_csv(csv_file, index=False)
print(f"Fichier CSV généré : {csv_file}")
# Affichage des résultats pour vérifier
#df_batch.show()

#df_election.show()  # Vérifier que les données sont bien chargées

Fichier CSV généré : /tmp/election_stream.csv


In [58]:
# Afficher les données pour vérifier le chargement
#df_election.show()

# 1. Ajouter une colonne "score_difference" au niveau des comtés
df_with_diff = df_batch.withColumn(
    "score_difference",
    F.abs(F.col("Trump") - F.col("Harris"))
)

# 2. Agréger les scores par État
df_state_aggregation = df_with_diff.groupBy("State").agg(
    F.sum("Trump").alias("total_trump"),
    F.sum("Harris").alias("total_harris")
)

# 3. Déterminer le candidat gagnant par État
df_state_with_leader = df_state_aggregation.withColumn(
    "leading_candidate",
    F.when(F.col("total_trump") > F.col("total_harris"), "Trump").otherwise("Harris")
)

# 4. Joindre les données agrégées par État au DataFrame initial
df_final = df_with_diff.join(
    df_state_with_leader,
    on="State",
    how="left"
)

# Vérification des données finales
#df_final.select(
#    "State", "County", "Trump", "Harris", 
#    "score_difference", "total_trump", 
#    "total_harris", "leading_candidate"
#).show()

#df_final.select(
#    "id", "County"
#).show()

# 5. Sauvegarder les données enrichies dans PostgreSQL
df_final.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://pg-ds-dellstore:5432/us_election_db") \
    .option("dbtable", "public.election_data_enriched") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .option("driver", "org.postgresql.Driver") \
    .mode("overwrite") \
    .save()

In [59]:
from kafka import KafkaProducer
import time
import csv
import json

producer = KafkaProducer(
    bootstrap_servers=['kafka:29092'],
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

# Lecture du fichier CSV
with open('/tmp/election_stream.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        # Envoi de chaque ligne à Kafka
        producer.send('election_stream', row)  # Envoi au topic Kafka
        print(f"Envoyé : {row}")
        time.sleep(2)  # Pause pour simuler un flux continu


Envoyé : {'id': '1', 'state': 'Alabama', 'county': 'Autauga', 'trump': '72.4', 'harris': '26.3'}
Envoyé : {'id': '3', 'state': 'Alabama', 'county': 'Barbour', 'trump': '56.9', 'harris': '42.2'}
Envoyé : {'id': '5', 'state': 'Alabama', 'county': 'Blount', 'trump': '90.0', 'harris': '9.1'}
Envoyé : {'id': '7', 'state': 'Alabama', 'county': 'Butler', 'trump': '61.0', 'harris': '38.3'}
Envoyé : {'id': '9', 'state': 'Alabama', 'county': 'Chambers', 'trump': '61.2', 'harris': '37.9'}
Envoyé : {'id': '11', 'state': 'Alabama', 'county': 'Chilton', 'trump': '85.6', 'harris': '13.7'}
Envoyé : {'id': '13', 'state': 'Alabama', 'county': 'Clarke', 'trump': '58.2', 'harris': '41.2'}
Envoyé : {'id': '15', 'state': 'Alabama', 'county': 'Cleburne', 'trump': '91.3', 'harris': '7.9'}
Envoyé : {'id': '17', 'state': 'Alabama', 'county': 'Colbert', 'trump': '72.6', 'harris': '26.3'}
Envoyé : {'id': '19', 'state': 'Alabama', 'county': 'Coosa', 'trump': '71.3', 'harris': '28.1'}
Envoyé : {'id': '21', 'state':

KeyboardInterrupt: 